# Neural network for Boston housing

In [ ]:
%matplotlib inline
import sklearn.datasets
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import tensorflow as tf
import seaborn as sns
from ml_course.helpers import sequential_dir

sns.set_context("poster")
plt.style.use('fivethirtyeight')
matplotlib.style.use('fivethirtyeight') 
plt.rcParams['figure.figsize'] = (10, 6)
scatter_size = 60

boston = sklearn.datasets.load_boston()

def feature_normalize(dataset):
    mu = np.mean(dataset,axis=0)
    sigma = np.std(dataset,axis=0)
    return (dataset - mu)/sigma

X = feature_normalize(boston.data)
Y = boston.target[:, np.newaxis]

In [ ]:
# Train-validation split
idx_train = np.random.rand(len(X)) < 0.8
idx_val = ~idx_train
X_train = X[idx_train]
Y_train = Y[idx_train]
X_val = X[idx_val]
Y_val = Y[idx_val]

In [ ]:
# Python variables
alpha = 0.01
learning_rate = 0.01
epochs = 500
dim_x = X.shape[1]
dim_y = Y.shape[1]
n_units = [dim_x, 20, 1]

# Computation graph
tf.reset_default_graph()

# Input
x = tf.placeholder(shape=(None, dim_x), dtype=tf.float32)
y = tf.placeholder(shape=(None, dim_y), dtype=tf.float32)
x_val = tf.placeholder(shape=(None, dim_x), dtype=tf.float32)
y_val = tf.placeholder(shape=(None, dim_y), dtype=tf.float32)

with tf.variable_scope('model'):
    # Layer 1
    with tf.variable_scope('layer1'):
        w1 = tf.get_variable("W", shape=(n_units[0], n_units[1]), initializer=random_init)
        b1 = tf.get_variable("b", shape=(n_units[1]), initializer=zero_init)
        z1 = tf.matmul(x, w1, name="z") + b1
        a1 = tf.tanh(z1)

    # Layer 2
    with tf.variable_scope('layer2'):
        w2 = tf.get_variable("W", shape=(n_units[1], n_units[2]), initializer=random_init)
        b2 = tf.get_variable("b", shape=(n_units[2]), initializer=zero_init)
        z2 = tf.add(tf.matmul(a1, w2), b2, name="z")
        y_ = z2    

# Evaluation and performance metrics
with tf.variable_scope('losses'):
    # regularization term
    # l2_pentalty = alpha * (tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2)
    mss_loss = tf.losses.mean_squared_error(y, y_)
    loss = mss_loss
    loss_train_summary = tf.summary.scalar('mss_train', loss)
    mss_val_summary = tf.summary.scalar('mss_val', mss_loss)
    mape = 100 * tf.reduce_mean(tf.abs(tf.div(y_ - y, y)))
    mape_train_summary = tf.summary.scalar('mape_train', mape)
    mape_val_summary = tf.summary.scalar('mape_val', mape)
    summary_train = tf.summary.merge([loss_train_summary, mape_train_summary])

# Trainin operators
with tf.variable_scope('train'): 
    random_init = tf.random_uniform_initializer(-0.1, 0.1, dtype=tf.float32)
    zero_init = tf.zeros_initializer(dtype=tf.float32)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train = optimizer.minimize(loss)
    global_init = tf.global_variables_initializer()

In [ ]:
sess = tf.InteractiveSession()
logdir = sequential_dir('log/boston_nn', dirname="2layers")
print('Storing log files in {}'.format(logdir))
filewriter = tf.summary.FileWriter(logdir, graph=sess.graph)
global_init.run()
y_.eval(feed_dict={x: X_train, y: Y_train})
for i in range(epochs):
    # Training plus summary
    _, train_loss, train_mape, summary_str_train = sess.run([train, loss, mape, summary_train], feed_dict={x: X_train, y: Y_train})
    filewriter.add_summary(summary_str_train, global_step=i)
    # Validation summary
    #summary_str_val = sess.run(summary_val, feed_dict={x: X_val, y: Y_val})
    #filewriter.add_summary(summary_str_val, global_step=i)
    # Print results to screen
    if (i+1)%100==0:
        print('epoch: {:4d} loss: {:5.3f} mape: {:3.1f}'.format(i, train_loss, train_mape))
sess.close()

## Tutorial Tasks

- Add an extra hidden layer with 10 units
- Visualize the validation error in TensorBoard
- Include regularization for all the layers